In [5]:
import pandas as pd


def normalize_trade_date(value):
    """Normalize any incoming date to a single string format '%m/%d/%Y'."""
    if value is None:
        return None

    # Try pandas first (handles many formats)
    try:
        dt = pd.to_datetime(value, errors="coerce")
        if not pd.isna(dt):
            return dt.strftime("%m/%d/%Y")
    except Exception:
        pass

    from datetime import datetime
    s = str(value).strip()
    if not s:
        return None

    for fmt in ("%Y-%m-%d", "%d/%m/%Y", "%m/%d/%Y", "%m-%d-%Y"):
        try:
            dt = datetime.strptime(s, fmt)
            return dt.strftime("%m/%d/%Y")
        except ValueError:
            continue

    return None


def generate_trades(csv_file, portfolio_type='simple'):
    """
    Generate trade code from CSV.
    
    Args:
        csv_file: Path to CSV file (must have: symbol, side, price, quantity columns; optional: date)
        portfolio_type: 'simple' or 'full'
    """
    df = pd.read_csv(csv_file)
    
    print("# Reset portfolio")
    print("reset_portfolio()\n")
    
    for _, row in df.iterrows():
        ticker = str(row['symbol']).lower()
        side = str(row['side']).lower()
        price = float(row['price'])
        qty = abs(float(row['quantity']))
        direction = str(row['direction']).lower()
        # Optional date column; normalize if present
        trade_date = None
        if 'date' in df.columns:
            trade_date = normalize_trade_date(row['date'])
        
        
        if portfolio_type == 'simple':
            if trade_date is not None:
                print(f"add_trade(ticker='{ticker}', action='{side}', direction='{direction}', price={price}, quantity_buy={qty}, date='{trade_date}')")
            else:
                print(f"add_trade(ticker='{ticker}', action='{side}', direction='{direction}', price={price}, quantity_buy={qty})")
        else:  # full
            if trade_date is not None:
                print(f"add_trade(ticker='{ticker}', asset_type='Equity', action='{side}', direction='{direction}', price={price}, quantity_buy={qty}, date='{trade_date}')")
            else:
                print(f"add_trade(ticker='{ticker}', asset_type='Equity', action='{side}', direction='{direction}', price={price}, quantity_buy={qty})")
    
    print("\n# Display")
    print("df = get_portfolio_df()")
    print("df")

In [6]:
# Example usage for portfolio_simple.ipynb
generate_trades('sample_trades.csv', 'simple')

# Reset portfolio
reset_portfolio()

add_trade(ticker='aapl', action='buy', direction='long', price=180.5, quantity_buy=10.0, date='01/02/2025')
add_trade(ticker='aapl', action='buy', direction='long', price=182.1, quantity_buy=5.0, date='01/03/2025')
add_trade(ticker='aapl', action='sell', direction='long', price=188.75, quantity_buy=8.0, date='01/10/2025')
add_trade(ticker='msft', action='buy', direction='long', price=420.0, quantity_buy=4.0, date='01/04/2025')
add_trade(ticker='msft', action='sell', direction='long', price=430.5, quantity_buy=2.0, date='01/08/2025')
add_trade(ticker='msft', action='sell', direction='long', price=435.2, quantity_buy=2.0, date='01/15/2025')
add_trade(ticker='btc', action='buy', direction='long', price=87000.0, quantity_buy=0.01, date='01/05/2025')
add_trade(ticker='btc', action='buy', direction='long', price=87500.0, quantity_buy=0.005, date='01/06/2025')
add_trade(ticker='btc', action='sell', direction='long', price=89000.0, quantity_buy=0.007, date=